---
# 신경망 종합 테스트

---

In [41]:
import numpy as np
import random
import nbimporter
import Encoder
import SP3
import TM

In [88]:
encoder = Encoder.ScalarEncoder(out_size=100, w=30, max_val=30)
sp = SP3.SpatialPooler(input_size=encoder.out_size, columns=512, minOver=5)
tm = TM.TemporalMemory(sdr_size=sp.columnCount)

In [106]:
for i in range(10):
    a = np.arange(30)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [91]:
for i in range(30):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sp.viz_activeCol()

Loading BokehJS ...

active columns : [31, 39, 51, 58, 66, 72, 109, 158, 321, 356, 368, 372, 390, 490] 
count : 14


Loading BokehJS ...

active columns : [6, 31, 34, 39, 72, 104, 109, 158, 191, 321, 322, 368, 372] 
count : 13


Loading BokehJS ...

...

active columns : [11, 26, 28, 73, 140, 193, 271, 276, 400, 408] 
count : 10


Loading BokehJS ...

active columns : [11, 28, 73, 119, 140, 193, 235, 342, 408, 429] 
count : 10


Loading BokehJS ...

KeyboardInterrupt: 

In [108]:
for i in range(29):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()

    sp.similarity(sdr1, sdr2)

Loading BokehJS ...

유사도 : 0.8181818181818182
[0, 35, 44, 86, 87, 222, 379, 386, 417, 463]
[6, 35, 44, 86, 87, 222, 379, 386, 417, 463]


Loading BokehJS ...

유사도 : 0.6666666666666666
[6, 35, 44, 86, 87, 222, 379, 386, 417, 463]
[6, 35, 44, 77, 87, 107, 379, 386, 417, 463]


Loading BokehJS ...

...

유사도 : 0.2777777777777778
[98, 141, 148, 175, 198, 241, 242, 327, 348, 407, 501]
[27, 53, 96, 141, 175, 198, 242, 255, 310, 348, 474, 483]


Loading BokehJS ...

유사도 : 0.3333333333333333
[27, 53, 96, 141, 175, 198, 242, 255, 310, 348, 474, 483]
[27, 111, 168, 172, 175, 188, 198, 241, 242, 310, 348, 437]


Loading BokehJS ...

유사도 : 0.29411764705882354
[27, 111, 168, 172, 175, 188, 198, 241, 242, 310, 348, 437]
[20, 27, 111, 168, 188, 231, 414, 420, 437, 442]


In [109]:
seq_data = [3,6,9]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat)
        sdr = sp.getActiveColumns()
        tm.compute(sdr)